In [ ]:

import numpy as np
import glob
import matplotlib.pyplot as plt
import scipy.linalg as sl
import math
import os
from multiprocess import Pool
from dynesty.utils import resample_equal


import nestle
import dynesty
from dynesty import plotting as dyplot
import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
import enterprise.signals.selections as selections


from enterprise import constants as const
from enterprise.signals import deterministic_signals
from enterprise.signals import gp_bases as gpb
from enterprise.signals import gp_priors as gpp
from enterprise.signals import (gp_signals, parameter, selections, utils,
                                white_signals)

from enterprise_extensions import deterministic as ee_deterministic

from enterprise_extensions import chromatic as chrom
from enterprise_extensions import dropout as drop
from enterprise_extensions import gp_kernels
from enterprise_extensions import model_orfs


from enterprise_extensions.blocks import (dm_noise_block,red_noise_block,chromatic_noise_block)
import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc


import time
st = time.time()


#PLEASE GIVE PULSAR NAME HERE


#PLEASE GIVE PULSAR NAME HERE

psrname = "J1909-3744"

#creating a txt file for saving output
#f= open("model_selection_"+psrname+".txt","w+")


#####Importing parfile and timfile#####



parfile = "../"+psrname+".NB.par"
timfile = "../"+psrname+".NB.tim"

psr = Pulsar(parfile, timfile,ephem='DE440')



tmin = psr.toas.min()
tmax = psr.toas.max()
Tspan = np.max(tmax) - np.min(tmin)
Tspan_chrom = 4 * Tspan
Tspan_years = Tspan/ 365.25 / 24 / 60 / 60
print(Tspan_years)  # time span of data in years



def by_groups(flags):
    """Selection function to split by -group flag"""
    flagvals = np.unique(flags["group"])
    return {val: flags["group"] == val for val in flagvals}

selection_by_groups = Selection(by_groups)


#Setting up Noise priors

#White noise
efac = parameter.Uniform(0.1, 8)
equad = parameter.Uniform(-8,-2)
ecorr = parameter.Uniform(-8, -2)

#White noise
ef = white_signals.MeasurementNoise(efac=efac, selection=selection_by_groups)
wn = white_signals.MeasurementNoise(efac=efac,log10_t2equad = equad, selection=selection_by_groups) 
ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection_by_groups)

# timing model
tm = gp_signals.TimingModel(use_svd=True, normed=True, coefficients=False)

#LET US DEFINE 4 DIFFERENT MODEL COMBINATIONS TO COMPARE THEIR EVIDENCES

model1 = ef + tm
model2 = wn + tm
model3 = ef + ec +  tm
model4 = wn + ec + tm


# initialize PTA

pta1 = signal_base.PTA([model1(psr)])
pta2 = signal_base.PTA([model2(psr)])
pta3 = signal_base.PTA([model3(psr)])
pta4 = signal_base.PTA([model4(psr)])


### PRIOR TRANSFORM, DYNESTY


def prior_transform_fn(pta):
    mins = np.array([param.prior.func_kwargs['pmin'] for param in pta.params])
    maxs = np.array([param.prior.func_kwargs['pmax'] for param in pta.params])
    spans = maxs-mins
    
    def prior_transform(cube):
        return spans*cube + mins
    
    return prior_transform

def dynesty_sample(pta):
    prior_transform = prior_transform_fn(pta)
    with  Pool() as pool:
        sampler = dynesty.NestedSampler(pta.get_lnlikelihood, prior_transform, len(pta.params),bound='multi',pool=pool,queue_size=os.cpu_count(), bootstrap = 0)
        sampler.run_nested(dlogz = 100, print_progress=False )
    res = sampler.results
    return res



In [ ]:

#Dynesty sampler run and plotting
Dres1 = dynesty_sample(pta1)
print("For model 1, log evidence = {} ± {}".format(Dres1.logz[-1], Dres1.logzerr[-1]))
weights1 = np.exp(Dres1['logwt'] - Dres1['logz'][-1])
samples1 = resample_equal(Dres1.samples, weights1)
fig1 = corner.corner(samples1,labels=list(pta1.param_names), label_kwargs={"fontsize": 7},
                     quantiles=(0.16, 0.5, 0.84),show_titles=True)
plt.savefig(psrname+'_model1.pdf')


In [ ]:
f= open("model_selection_"+psrname+".txt","w+")

In [14]:
x = 2

In [18]:
f= open("model_selection_"+psrname+".txt","w+")


In [22]:
print('Hello, Python!', file = f)


In [23]:
f.close()

In [ ]:


#Dynesty sampler run and plotting
Dres1 = dynesty_sample(pta1)
print("For model 1, log evidence = {} ± {}".format(Dres1.logz[-1], Dres1.logzerr[-1]), file = f)
weights1 = np.exp(Dres1['logwt'] - Dres1['logz'][-1])
samples1 = resample_equal(Dres1.samples, weights1)
fig1 = corner.corner(samples1,labels=list(pta1.param_names), label_kwargs={"fontsize": 7},
                     quantiles=(0.16, 0.5, 0.84),show_titles=True)
plt.savefig(psrname+'_model1.pdf')



Process ForkPoolWorker-26:
Process ForkPoolWorker-32:
Process ForkPoolWorker-20:
Process ForkPoolWorker-25:
Process ForkPoolWorker-27:
Process ForkPoolWorker-30:
Process ForkPoolWorker-29:
Process ForkPoolWorker-21:
Process ForkPoolWorker-28:
Process ForkPoolWorker-19:
Process ForkPoolWorker-17:
Process ForkPoolWorker-22:
Process ForkPoolWorker-24:
Process ForkPoolWorker-31:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-23:
Process ForkPoolWorker-18:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/aman/anaconda3/envs/pulsar/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/home/aman/anaconda3/envs/pulsar/lib/python3.9/site-packages/multiprocess/process.py", line 315, in _bootstrap
    self.run()
  File "/home/aman/a

  File "/home/aman/anaconda3/envs/pulsar/lib/python3.9/site-packages/multiprocess/queues.py", line 367, in get
    with self._rlock:
  File "/home/aman/anaconda3/envs/pulsar/lib/python3.9/site-packages/multiprocess/queues.py", line 367, in get
    with self._rlock:
  File "/home/aman/anaconda3/envs/pulsar/lib/python3.9/site-packages/multiprocess/synchronize.py", line 101, in __enter__
    return self._semlock.__enter__()
  File "/home/aman/anaconda3/envs/pulsar/lib/python3.9/site-packages/multiprocess/synchronize.py", line 101, in __enter__
    return self._semlock.__enter__()
  File "/home/aman/anaconda3/envs/pulsar/lib/python3.9/site-packages/multiprocess/synchronize.py", line 101, in __enter__
    return self._semlock.__enter__()
  File "/home/aman/anaconda3/envs/pulsar/lib/python3.9/site-packages/multiprocess/pool.py", line 114, in worker
    task = get()
  File "/home/aman/anaconda3/envs/pulsar/lib/python3.9/site-packages/multiprocess/pool.py", line 114, in worker
    task = get()